In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/final main.csv")
df.shape

(66397, 2)

In [12]:
df.shape

(66397, 2)

In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66397 entries, 0 to 66396
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_info  66397 non-null  object
 1   desc          66397 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [16]:

import numpy as np
import re
import itertools
from collections import Counter
import pandas as pd
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


In [17]:
for i in range(0,df.shape[0]):
  df['desc'][i]= clean_str(df['desc'][i])

In [18]:
df.head()

,product_info,desc
0,SEER SECRETS ACTIVE SILVER ION DEODORANT CREAM,1mg body lotion with shea butter vitamin e p 1...
1,SEER SECRETS MILK BODY LOTION SPF 21,1mg body lotion with shea butter vitamin e p 1...
2,VITRO NATURALS ALOE SHEA BODY BUTTER,1mg body lotion with shea butter vitamin e p 1...
3,1MILE HEALTHCARE CLASSIC TONE STETHOSCOPE,1mile healthcare classic tone stethoscope orde...
4,1MILE HEALTHCARE DOCTOR PLUS STETHOSCOPE,1mile healthcare doctor plus stethoscope 606 o...


In [21]:
documents = df['desc'].values.astype("U")

In [22]:
vectorizer = TfidfVectorizer(stop_words='english')
features = vectorizer.fit_transform(documents)

In [ ]:
# #ELBOW METHOD TO FIND K
# cost =[]
# for i in range(1000, 5000,500):
#     KM = KMeans(n_clusters = i, max_iter = 500)
#     KM.fit(features)
#     cost.append(KM.inertia_)     
  
# plt.plot(range(1000,5000), cost, color ='g', linewidth ='3')
# plt.xlabel("Value of K")
# plt.ylabel("Sqaured Error (Cost)")
# plt.show() 
  


In [23]:
k = 1000
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=1000, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [25]:
import pickle
pickle.dump(model, open("/content/drive/My Drive/KMEANS CLUTERING with TF-IDF Vectorizer.sav", 'wb'))

In [38]:
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)


In [40]:
model= pickle.load(open("/content/drive/My Drive/KMEANS CLUTERING with TF-IDF Vectorizer.sav", 'rb'))

In [41]:
df['cluster'] = model.labels_

In [42]:
df['cluster']

0         17
1         17
2         17
3        586
4        586
        ... 
66392    556
66393     83
66394    163
66395    460
66396     83
Name: cluster, Length: 66397, dtype: int32

In [44]:
clusters.groups

In [45]:
for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w') 
    data = clusters.get_group(cluster)[['product_info','desc']] 
    f.write(data.to_csv(index_label='id')) 
    f.close()

print("Cluster centroids: \n")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(k):
    print("Cluster %d:" % i)
    for j in order_centroids[i, :10]:
        print (' %s' % terms[j])
    print('------------')

Streaming output truncated to the last 5000 lines.
 mattress
 protector
 waterproof
 dry
 140cm
 mat
 adults
------------
Cluster 584:
 165
 157
 lm
 20
 bought
 mrp
 globules
 sbl
 bottle
 gm
------------
Cluster 585:
 insta
 hydrate
 scoop
 protein
 pineapple
 orange
 ample
 lets
 energy
 speeds
------------
Cluster 586:
 price
 capsule
 online
 best
 know
 specifications
 1mg
 tablet
 information
 india
------------
Cluster 587:
 gerber
 113gm
 sitter
 2nd
 food
 apple
 store
 pear
 banana
 juice
------------
Cluster 588:
 aukris
 belt
 lumbo
 sacral
 blue
 contoured
 medium
 small
 price
 box
------------
Cluster 589:
 butternut
 butter
 unsweetened
 peanut
 honey
 creamy
 crunchy
 almond
 kg
 jar
------------
Cluster 590:
 iron
 folic
 acid
 ranging
 elemental
 nucleotide
 numerous
 bodily
 biosynthesis
 energetic
------------
Cluster 591:
 glycerinum
 bjain
 globules
 ch
 price
 dilution
 30
 otc441940order
 bottle
 know
------------
Cluster 592:
 creatine
 monohydrate
 myprotein

In [32]:
new_doc = ["sanitary pads"]
Y = vectorizer.transform(new_doc)
prediction = model.predict(Y)
print(prediction)

[83]


In [33]:
result=pd.read_csv("cluster{}.csv".format(prediction[0]))

In [36]:
for i  in result["product_info"]:
    print(i)

Streaming output truncated to the last 5000 lines.
INLIFE MULTIVITAMINS AND MINERALS CAPSULE MEN
INLIFE POWERFUL SURFACE CLEANER
INLIFE VITALITY AND VIGOUR VEGETARIAN CAPSULES
INSTAFREE 72 TABLET
INSTALIB CAPSULE
INSUPEN EZ PEN
INSUPEN EZ PEN
FDSON 12 TABLET
VWASH PLUS INTIMATE HYGIENE WIPES
FERANID TABLET
IONAX T SCALP SOLUTION
ISHA SURGICAL SITZ/PILES BATH TUB
NEW I SITE CAPSULE
ISOFLAV CR CAPSULE
STERI VEGETABLES AND FRUITS WASH LIQUID
ITIS EYE DROP
GENCOCAL TABLET
JAFIX PLUS TABLET
GENCOCAL TABLET
JAIN KAISHORE GUGGULU TABLET
JAIN MAHAYOGARAJ GUGGULU TABLET
JAIN METHI POWDER
JAIN TRIPHALA GUGGULU TABLET
JAIN FEMITAB CAPSULE
JAIN KAISHORE GUGGULU TABLET
JAIN MAHAYOGARAJ GUGGULU TABLET
JAIN METHI POWDER
JAIN TRIPHALA GUGGULU TABLET
JIVA ARJUNA TABLET
JIVA ASHWAGANDHA TABLET
JIVA AYURVEDIC TEA
JIVA DASHMOOL KWATH DRY
JIVA DIATRIN TABLET
JIVA ROSE WATER
JIVA ARJUNA TABLET
JIVA ASHWAGANDHA TABLET
JIVA AYURVEDIC TEA
JIVA DIATRIN TABLET
JIVA EQUILINE TABLET
JIVA HONEY
JIVA ROSE PETAL WATE